In [ ]:
from pynq import Overlay
from pynq import DefaultIP
import pynq.lib.dma

class AddDriver(DefaultIP):
    def __init__(self,description):
        super().__init__(description=description)
    
    bindto = ['xilinx.com:hls:add:1.0']
    
    def add(self,a,b):
        self.write(0x10,a)
        self.write(0x18,b)
        return self.read(0x20)
    
class ConstantMultiplyDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)

    bindto = ['xilinx.com:hls:vector_multiplier:1.0']

    @property
    def constant(self):
        return self.read(0x10)

    @constant.setter
    def constant(self, value):
        self.write(0x10, value)
        
    def constant_list(self,value):
        for i in range(len(value)):
            self.write(0x3f + (i*4),value[i])
            
# from pynq import DefaultHierarchy

# class StreamMultiplyDriver(DefaultHierarchy):
#     def __init__(self, description):
#         super().__init__(description)

#     def stream_multiply(self, stream, constant):
#         self.multiply.constant = constant
#         with xlnk.cma_array(shape=(len(stream),), dtype=np.uint32) as in_buffer,\
#              xlnk.cma_array(shape=(len(stream),), dtype=np.uint32) as out_buffer:
#             for i, v, in enumerate(stream):
#                 in_buffer[i] = v
#             self.multiply_dma.sendchannel.transfer(in_buffer)
#             self.multiply_dma.recvchannel.transfer(out_buffer)
#             self.multiply_dma.sendchannel.wait()
#             self.multiply_dma.recvchannel.wait()
#             result = out_buffer.copy()
#         return result

#     @staticmethod
#     def checkhierarchy(description):
#         if 'multiply_dma' in description['ip'] and 'multiply' in description['ip']:
#             return True
#         return False

In [62]:
ol = Overlay('/home/xilinx/jupyter_notebooks/Praeklima /testing/vector_multiplier.bit')
dma = ol.custom_multiplier.axi_dma_0
multiply = ol.custom_multiplier.vector_multiplier
ol.custom_multiplier.vector_multiplier
ol?
# ol.ip_dict
# ol.hierarchy_dict

Type:            Overlay
String form:     <pynq.overlay.Overlay object at 0x7f8b193be0>
File:            /usr/local/lib/python3.6/dist-packages/pynq/overlay.py
Docstring:      
Default documentation for overlay /home/xilinx/jupyter_notebooks/Praeklima /testing/vector_multiplier.bit. The following
attributes are available on this overlay:

IP Blocks
----------
custom_multiplier/vector_multiplier : __main__.ConstantMultiplyDriver
adder                : __main__.AddDriver
custom_multiplier/axi_dma_0 : pynq.lib.dma.DMA
zynq_ultra_ps_e_0    : pynq.overlay.DefaultIP

Hierarchies
-----------
custom_multiplier    : pynq.overlay.DefaultHierarchy

Interrupts
----------
None

GPIO Outputs
------------
None

Memories
------------
zynq_ultra_ps_e_0    : Memory
Class docstring:
This class keeps track of a single bitstream's state and contents.

The overlay class holds the state of the bitstream and enables run-time
protection of bindings.

Our definition of overlay is: "post-bitstream configurable d

In [63]:
from pynq import Xlnk
import numpy as np

xlnk = Xlnk()
in_buffer = xlnk.cma_array(shape=(5,), dtype=np.uint32)
out_buffer = xlnk.cma_array(shape=(5,), dtype=np.uint32)

for i in range(5):
    in_buffer[i] = i*5
# x = [1,2,3,4,5]
multiply.constant = 5
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()

out_buffer

/usr/lib/python3/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: pynq.Xlnk is deprecated and will be removed in 2.7 - use pynq.allocate instead
  after removing the cwd from sys.path.


PynqBuffer([  0,  25,  50,  75, 100], dtype=uint32)

In [ ]:
from pynq import DefaultHierarchy

class StreamMultiplyDriver(DefaultHierarchy):
    def __init__(self, description):
        super().__init__(description)

    def stream_multiply(self, stream, constant):
        self.multiply.constant = constant
        with xlnk.cma_array(shape=(len(stream),), dtype=np.uint32) as in_buffer,\
             xlnk.cma_array(shape=(len(stream),), dtype=np.uint32) as out_buffer:
            for i, v, in enumerate(stream):
                in_buffer[i] = v
            self.multiply_dma.sendchannel.transfer(in_buffer)
            self.multiply_dma.recvchannel.transfer(out_buffer)
            self.multiply_dma.sendchannel.wait()
            self.multiply_dma.recvchannel.wait()
            result = out_buffer.copy()
        return result

    @staticmethod
    def checkhierarchy(description):
        if 'multiply_dma' in description['ip'] and 'multiply' in description['ip']:
            return True
        return False

In [ ]:
# stream_multiply(in_buffer,5)
ol?